In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.graph_objects as go
import streamlit as st

In [2]:
# Leitura dos dados
file = "../data/run_data.csv"
colnames = ['data', 'quilometragem', 'tempo_min', 'calorias']
df = pd.read_csv(file, names=colnames)

In [3]:
# Função para filtrar os dados ignorando valores 0
def filtrar_dados(df):
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%y')
    df['quilometragem'] = df['quilometragem'].astype(str).str.replace(',', '.').astype(float)
    df['dia_semana'] = df['data'].dt.day_name()
    df_filtrado = df[(df['quilometragem'] > 0.0)]
    return df_filtrado.copy()

In [4]:
media_calorias_por_minuto = (df['calorias'] / df['tempo_min']).mean()

In [5]:
media_calorias_por_minuto

7.141150773957512

In [4]:
df.tail()

,data,quilometragem,tempo_min,calorias
53,30/07/24,"4,6",41,302
54,31/07/24,"5,1",45,332
55,01/08/24,0,0,0
56,02/08/24,0,0,0
57,03/08/24,0,0,0


In [5]:
# Aplicar a função para filtrar os dados
df_filtrado = filtrar_dados(df)

In [6]:
# Função para calcular médias
def calcular_medias(df, df_filtrado):
    media_calorias_com_zeros = df['calorias'].mean()
    media_calorias_sem_zeros = df_filtrado['calorias'].mean()
    media_km = df_filtrado['quilometragem'].mean()
    media_cal = df_filtrado['calorias'].mean()
    media_calorias_por_minuto = df_filtrado['calorias_por_minuto'].mean()
    media_km_por_minuto = df_filtrado['km_por_minuto'].mean()
    return media_calorias_com_zeros, media_calorias_sem_zeros, media_km, media_cal, media_calorias_por_minuto, media_km_por_minuto


In [9]:
# Função para calcular métricas e adicionar colunas ao DataFrame
def calcular_metricas(df):
    df['calorias_por_minuto'] = df['calorias'] / df['tempo_min']
    df['km_por_minuto'] = df['quilometragem'] / (df['tempo_min'] / 60)
    return df


In [10]:
# Função para calcular médias por dia da semana
def calcular_medias_por_dia_semana(df_filtrado):
    df_semana_calorias = df_filtrado[['dia_semana', 'calorias_por_minuto']]
    media_calorias_por_dia_semana = df_semana_calorias.groupby('dia_semana')['calorias_por_minuto'].mean().sort_values(ascending=False)
    
    df_semana_km = df_filtrado[['dia_semana', 'km_por_minuto']]
    media_km_por_dia_semana = df_semana_km.groupby('dia_semana')['km_por_minuto'].mean().sort_values(ascending=False)
    
    return media_calorias_por_dia_semana, media_km_por_dia_semana

In [11]:
# Calcular métricas
df_filtrado = calcular_metricas(df_filtrado)

# Calcular médias
media_calorias_com_zeros, media_calorias_sem_zeros, media_km, media_cal, media_calorias_por_minuto, media_km_por_minuto = calcular_medias(df, df_filtrado)

# Calcular médias por dia da semana
media_calorias_por_dia_semana, media_km_por_dia_semana = calcular_medias_por_dia_semana(df_filtrado)


In [12]:
# Função para calcular médias
def calcular_medias(df, df_filtrado):
    media_calorias_com_zeros = df['calorias'].mean()
    media_calorias_sem_zeros = df_filtrado['calorias'].mean()
    return media_calorias_com_zeros, media_calorias_sem_zeros

In [38]:
def criar_grafico(df, media_calorias_com_zeros, media_calorias_sem_zeros):
    # Criar o gráfico de dispersão
    fig = go.Figure()

    # Adicionar a linha de calorias (linha conectando os pontos)
    fig.add_trace(go.Scatter(
        x=df['data'],
        y=df['calorias'],
        mode='lines+markers',  # Adiciona linha e marcadores
        line=dict(color='white'),
        marker=dict(
            color=['green' if calorias > media_calorias_sem_zeros else 'red' for calorias in df['calorias']],
            size=10
        ),
        text=df.apply(lambda row: f"Calorias: {row['calorias']}<br>Quilometragem: {row['quilometragem']}", axis=1),
        hoverinfo='text',
        name='Calorias'
    ))

    # Adicionar linha da média de calorias sem zeros
    fig.add_trace(go.Scatter(
        x=[df['data'].min(), df['data'].max()],
        y=[media_calorias_sem_zeros, media_calorias_sem_zeros],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name=f'Média de Calorias (sem zeros: {media_calorias_sem_zeros:.2f})'
    ))

    # Adicionar linha da média de calorias com zeros
    fig.add_trace(go.Scatter(
        x=[df['data'].min(), df['data'].max()],
        y=[media_calorias_com_zeros, media_calorias_com_zeros],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name=f'Média de Calorias (com zeros: {media_calorias_com_zeros:.2f})'
    ))

    # Adicionar os rótulos com data e dia da semana
    fig.update_xaxes(
        tickmode='array',
        tickvals=df['data'],
        ticktext=df['data'].dt.strftime('%d-%b-%a'),  # Formato dia-mês-abreviação do dia da semana
        tickangle=-90,  # Inclinação dos rótulos para melhorar a legibilidade
        tickfont=dict(size=10)  # Tamanho da fonte dos rótulos
    )

    # Personalizar o layout
    fig.update_layout(
        title='Calorias por Dia com Médias',
        xaxis_title='Data',
        yaxis_title='Calorias',
        plot_bgcolor='lightgray',  # Fundo do gráfico cinza
        paper_bgcolor='lightgray',  # Fundo da página cinza
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=150)  # Ajustar as margens para dar mais espaço para os rótulos do eixo x
    )

    # Mostrar o gráfico 
    fig.show()


In [39]:
criar_grafico(df, media_calorias_com_zeros, media_calorias_sem_zeros)